<a href="https://colab.research.google.com/github/pw0908/Caltech-CS155-2022/blob/main/sets/set5/set5_prob3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Set 5
## 3. Word2Vec \*\*Principles**

#### Preparation

In [1]:
# download the helper function
!wget -O P3CHelpers.py https://raw.githubusercontent.com/charlesincharge/Caltech-CS155-2022/main/sets/set5/P3CHelpers.py

--2022-02-15 22:03:34--  https://raw.githubusercontent.com/charlesincharge/Caltech-CS155-2022/main/sets/set5/P3CHelpers.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4939 (4.8K) [text/plain]
Saving to: ‘P3CHelpers.py’

P3CHelpers.py       100%[===================>]   4.82K  --.-KB/s    in 0.001s  

2022-02-15 22:03:34 (4.92 MB/s) - ‘P3CHelpers.py’ saved [4939/4939]



In [2]:
# download the dataset
!wget -O dr_seuss.txt https://raw.githubusercontent.com/charlesincharge/Caltech-CS155-2022/main/sets/set5/data/dr_seuss.txt

--2022-02-15 22:03:37--  https://raw.githubusercontent.com/charlesincharge/Caltech-CS155-2022/main/sets/set5/data/dr_seuss.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8810 (8.6K) [text/plain]
Saving to: ‘dr_seuss.txt’

dr_seuss.txt        100%[===================>]   8.60K  --.-KB/s    in 0s      

2022-02-15 22:03:38 (24.6 MB/s) - ‘dr_seuss.txt’ saved [8810/8810]



In [8]:
import numpy as np
from P3CHelpers import *
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset

#### Problem D: 
Fill in the generate_traindata and find_most_similar_pairs functions.

In [25]:
def get_word_repr(word_to_index, word):
    """
    Returns one-hot-encoded feature representation of the specified word given
    a dictionary mapping words to their one-hot-encoded index.

    Arguments:
        word_to_index: Dictionary mapping words to their corresponding index
                       in a one-hot-encoded representation of our corpus.

        word:          Word whose feature representation we wish to compute.

    Returns:
        feature_representation:     Feature representation of the passed-in word.
    """
    unique_words = word_to_index.keys()
    # Return a vector that's zero everywhere besides the index corresponding to <word>
    feature_representation = np.zeros(len(unique_words))
    feature_representation[word_to_index[word]] = 1
    return feature_representation    

def generate_traindata(word_list, word_to_index, window_size=4):
    """
    Generates training data for Skipgram model.

    Arguments:
        word_list:     Sequential list of words (strings).
        word_to_index: Dictionary mapping words to their corresponding index
                       in a one-hot-encoded representation of our corpus.

        window_size:   Size of Skipgram window. Defaults to 2 
                       (use the default value when running your code).

    Returns:
        (trainX, trainY):     A pair of matrices (trainX, trainY) containing training 
                              points (one-hot-encoded vectors) and their corresponding output_word
                              (also one-hot-encoded vectors)

    """
    trainX = []
    trainY = []

    for i in range(len(word_list)):
        x = get_word_repr(word_to_index, word_list[i])
        for j in range(i-window_size,i+window_size+1):
            if j>=0 and j<len(word_list):
                trainX.append(x)
                y = get_word_repr(word_to_index, word_list[j])
                trainY.append(y)
    return np.array(trainX), np.array(trainY)

class word2vectorDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X).float()
        self.y = torch.tensor(np.argmax(y, axis = 1)).long()
    def __len__(self):
        return self.X.shape[0]
    def __getitem__(self, index):  
          return self.X[index,:], self.y[index]

In [32]:
def find_most_similar_pairs(filename, num_latent_factors):
    """
    Find the most similar pairs from the word embeddings computed from
    a body of text
    
    Arguments:
        filename:           Text file to read and train embeddings from
        num_latent_factors: The number of latent factors / the size of the embedding
    """
    # Load in a list of words from the specified file; remove non-alphanumeric characters
    # and make all chars lowercase.
    sample_text = load_word_list(filename)

    # Create word dictionary
    word_to_index = generate_onehot_dict(sample_text)
    
    # Create training data
    trainX, trainY = generate_traindata(sample_text, word_to_index)
    train_dataset = word2vectorDataset(trainX, trainY)
    print(len(word_to_index))
    vocab_size = len(word_to_index)
    model = nn.Sequential(nn.Linear(vocab_size, 10),nn.Linear(10, vocab_size),)

    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    loss_fn = nn.CrossEntropyLoss()

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)

    for epoch in range(20):
        for batch_idx, (data, target) in enumerate(train_loader):
            optimizer.zero_grad()
            output = model(data)
            loss = loss_fn(output, target)
            loss.backward()
            optimizer.step()
    weights = list(model.parameters())[2].detach().numpy()
    print(weights.shape)
    # Find and print most similar pairs
    similar_pairs = most_similar_pairs(weights, word_to_index)
    for pair in similar_pairs[:30]:
        print(pair)

### Problem E-H:
Run your model on drseuss.txt and answer questions from E through H.

In [36]:
find_most_similar_pairs('dr_seuss.txt', 10)

308
(308, 10)
Pair(eight, nine), Similarity: 0.98720694
Pair(nine, eight), Similarity: 0.98720694
Pair(dad, zeds), Similarity: 0.96925426
Pair(zeds, dad), Similarity: 0.96925426
Pair(he, drink), Similarity: 0.9666724
Pair(drink, he), Similarity: 0.9666724
Pair(likes, drink), Similarity: 0.96280104
Pair(shoe, off), Similarity: 0.96162623
Pair(off, shoe), Similarity: 0.96162623
Pair(upon, heads), Similarity: 0.96127015
Pair(heads, upon), Similarity: 0.96127015
Pair(thin, four), Similarity: 0.95897
Pair(four, thin), Similarity: 0.95897
Pair(ink, drink), Similarity: 0.9581423
Pair(yet, wire), Similarity: 0.9571183
Pair(wire, yet), Similarity: 0.9571183
Pair(these, pets), Similarity: 0.95471144
Pair(pets, these), Similarity: 0.95471144
Pair(kind, time), Similarity: 0.95429236
Pair(time, kind), Similarity: 0.95429236
Pair(cut, haircut), Similarity: 0.953977
Pair(haircut, cut), Similarity: 0.953977
Pair(brush, comb), Similarity: 0.9531305
Pair(comb, brush), Similarity: 0.9531305
Pair(fly, yet